In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

2022-12-07 19:33:57.934118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 19:33:58.058243: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-12-07 19:33:58.058263: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-07 19:33:59.611699: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No s

In [2]:
random_seed = 69
np.random.seed(random_seed)

df = pd.read_csv('munster_hourly.csv', parse_dates=["DATUM"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35672 entries, 0 to 35671
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATUM        35672 non-null  datetime64[ns]
 1   STATIONS_ID  35672 non-null  int64         
 2   QN_4         35672 non-null  int64         
 3   TT_TER       35672 non-null  float64       
 4   RF_TER       35672 non-null  float64       
 5   VGSL         35672 non-null  float64       
 6   TS05         35672 non-null  float64       
 7   BF10         35672 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 2.2 MB


In [3]:
df = df[["DATUM", "BF10"]]
df['BF10'] = df["BF10"].astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35672 entries, 0 to 35671
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATUM   35672 non-null  datetime64[ns]
 1   BF10    35672 non-null  float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 418.2 KB


In [14]:
TIME_WINDOW=100
FORECAST_DISTANCE=30

targetcol = "BF10"
targetcol_series = df[targetcol]

X_train, X_else, targetcol_series_train, targetcol_series_else = train_test_split(df, 
                                   targetcol_series, 
                                   test_size=0.2, 
                                   shuffle=False)
X_valid, X_test, targetcol_series_valid, targetcol_series_test = train_test_split(X_else, targetcol_series_else, test_size=0.5, shuffle=False)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print()
print(targetcol_series_train.shape)
print(targetcol_series_valid.shape)
print(targetcol_series_test.shape)

(28537, 2)
(3567, 2)
(3568, 2)

(28537,)
(3567,)
(3568,)


In [15]:
def sliding_window_2D_from_df(df, targetcol_series, TIME_WINDOW, FORECAST_DISTANCE):
    X_df = pd.concat([df.shift(i) for i in range(1,TIME_WINDOW+1)], axis=1)\
                      .iloc[TIME_WINDOW : len(df) - FORECAST_DISTANCE + 1]
    X_df.columns = pd.MultiIndex.from_product([range(1,TIME_WINDOW+1), df.columns])
    X_df.rename_axis(["lag", "variable"], axis=1, inplace=True)

    y_series = targetcol_series.shift( -FORECAST_DISTANCE + 1 )\
                      .iloc[ TIME_WINDOW : len(df) - FORECAST_DISTANCE + 1 ]
    return X_df, y_series

In [16]:
def spot_check_sliding_window_2D(X_df, y_series, df, target_series, FORECAST_DISTANCE=1):
    TIME_WINDOW = X_df.shape[1] // df.shape[1]
    display(X_df.head(1).iloc[:,[*list(range(2*df.shape[1])), *list(range(-2*df.shape[1],0))]])
    print("The following should be the first two lagged values:")
    display(df.iloc[:TIME_WINDOW].tail(2))
    print("The following should be the last two lagged values:")
    display(df.head(2))

    print(f"\nThe target (FORECAST_DISTANCE={FORECAST_DISTANCE}):")
    display(y_series.iloc[:10].to_frame().T)
    print(f"The actual values from the end of the first time window (TIME_WINDOW={TIME_WINDOW}):")
    display(target_series.iloc[TIME_WINDOW:].iloc[:FORECAST_DISTANCE+9].to_frame().T)

In [17]:
X_df_train, y_train = sliding_window_2D_from_df(X_train, targetcol_series_train, TIME_WINDOW, FORECAST_DISTANCE)
X_df_valid, y_valid = sliding_window_2D_from_df(X_valid, targetcol_series_valid, TIME_WINDOW, FORECAST_DISTANCE)
X_df_test, y_test = sliding_window_2D_from_df(X_test, targetcol_series_test, TIME_WINDOW, FORECAST_DISTANCE)

In [18]:
spot_check_sliding_window_2D(X_df_train, y_train, X_train, targetcol_series_train, FORECAST_DISTANCE)

lag                      1                         2          \
variable               DATUM  BF10               DATUM  BF10   
100      1991-02-03 07:00:00  97.0 1991-02-02 21:00:00  97.0   

lag                      99                         100         
variable               DATUM   BF10               DATUM   BF10  
100      1991-01-01 14:00:00  102.0 1991-01-01 07:00:00  102.0

The following should be the first two lagged values:


,DATUM,BF10
98,1991-02-02 21:00:00,97.0
99,1991-02-03 07:00:00,97.0


The following should be the last two lagged values:


,DATUM,BF10
0,1991-01-01 07:00:00,102.0
1,1991-01-01 14:00:00,102.0



The target (FORECAST_DISTANCE=30):


,100,101,102,103,104,105,106,107,108,109
BF10,99.0,99.0,99.0,100.0,100.0,100.0,106.0,106.0,106.0,102.0


The actual values from the end of the first time window (TIME_WINDOW=100):


,100,101,102,103,104,105,106,107,108,109,...,129,130,131,132,133,134,135,136,137,138
BF10,97.0,97.0,97.0,97.0,97.0,96.0,96.0,96.0,96.0,96.0,...,99.0,99.0,99.0,100.0,100.0,100.0,106.0,106.0,106.0,102.0


In [19]:
## where we are at...: something we can feed into generic models (e.g., feedfwd NN's, sklearn models like RF...)
X_df_train

lag                      1                          2           \
variable               DATUM   BF10               DATUM   BF10   
100      1991-02-03 07:00:00   97.0 1991-02-02 21:00:00   97.0   
101      1991-02-03 14:00:00   97.0 1991-02-03 07:00:00   97.0   
102      1991-02-03 21:00:00   97.0 1991-02-03 14:00:00   97.0   
103      1991-02-04 07:00:00   97.0 1991-02-03 21:00:00   97.0   
104      1991-02-04 14:00:00   97.0 1991-02-04 07:00:00   97.0   
...                      ...    ...                 ...    ...   
28503    2017-01-04 18:00:00  109.0 2017-01-04 12:00:00  109.0   
28504    2017-01-05 06:00:00  104.0 2017-01-04 18:00:00  109.0   
28505    2017-01-05 12:00:00  104.0 2017-01-05 06:00:00  104.0   
28506    2017-01-05 18:00:00  104.0 2017-01-05 12:00:00  104.0   
28507    2017-01-06 06:00:00  104.0 2017-01-05 18:00:00  104.0   

lag                      3                          4           \
variable               DATUM   BF10               DATUM   BF10   
100      1991-02-02 14:00:00   97.0 1991-02-02 07:00:00   97.0   
101      1991-02-02 21:00:00   97.0 1991-02-02 14:00:00   97.0   
102      1991-02-03 07:00:00   97.0 1991-02-02 21:00:00   97.0   
103      1991-02-03 14:00:00   97.0 1991-02-03 07:00:00   97.0   
104      1991-02-03 21:00:00   97.0 1991-02-03 14:00:00   97.0   
...                      ...    ...                 ...    ...   
28503    2017-01-04 06:00:00  109.0 2017-01-03 18:00:00  102.0   
28504    2017-01-04 12:00:00  109.0 2017-01-04 06:00:00  109.0   
28505    2017-01-04 18:00:00  109.0 2017-01-04 12:00:00  109.0   
28506    2017-01-05 06:00:00  104.0 2017-01-04 18:00:00  109.0   
28507    2017-01-05 12:00:00  104.0 2017-01-05 06:00:00  104.0   

lag                      5           ...                 96          \
variable               DATUM   BF10  ...               DATUM   BF10   
100      1991-02-01 21:00:00   97.0  ... 1991-01-02 14:00:00  110.0   
101      1991-02-02 07:00:00   97.0  ... 1991-01-02 21:00:00  110.0   
102      1991-02-02 14:00:00   97.0  ... 1991-01-03 07:00:00  110.0   
103      1991-02-02 21:00:00   97.0  ... 1991-01-03 14:00:00  110.0   
104      1991-02-03 07:00:00   97.0  ... 1991-01-03 21:00:00  110.0   
...                      ...    ...  ...                 ...    ...   
28503    2017-01-03 12:00:00  102.0  ... 2016-12-04 06:00:00   97.0   
28504    2017-01-03 18:00:00  102.0  ... 2016-12-04 12:00:00   97.0   
28505    2017-01-04 06:00:00  109.0  ... 2016-12-04 18:00:00   97.0   
28506    2017-01-04 12:00:00  109.0  ... 2016-12-05 06:00:00   97.0   
28507    2017-01-04 18:00:00  109.0  ... 2016-12-05 12:00:00   97.0   

lag                      97                         98          \
variable               DATUM   BF10               DATUM   BF10   
100      1991-01-02 07:00:00  110.0 1991-01-01 21:00:00  102.0   
101      1991-01-02 14:00:00  110.0 1991-01-02 07:00:00  110.0   
102      1991-01-02 21:00:00  110.0 1991-01-02 14:00:00  110.0   
103      1991-01-03 07:00:00  110.0 1991-01-02 21:00:00  110.0   
104      1991-01-03 14:00:00  110.0 1991-01-03 07:00:00  110.0   
...                      ...    ...                 ...    ...   
28503    2016-12-03 18:00:00   98.0 2016-12-03 12:00:00   98.0   
28504    2016-12-04 06:00:00   97.0 2016-12-03 18:00:00   98.0   
28505    2016-12-04 12:00:00   97.0 2016-12-04 06:00:00   97.0   
28506    2016-12-04 18:00:00   97.0 2016-12-04 12:00:00   97.0   
28507    2016-12-05 06:00:00   97.0 2016-12-04 18:00:00   97.0   

lag                      99                         100         
variable               DATUM   BF10               DATUM   BF10  
100      1991-01-01 14:00:00  102.0 1991-01-01 07:00:00  102.0  
101      1991-01-01 21:00:00  102.0 1991-01-01 14:00:00  102.0  
102      1991-01-02 07:00:00  110.0 1991-01-01 21:00:00  102.0  
103      1991-01-02 14:00:00  110.0 1991-01-02 07:00:00  110.0  
104      1991-01-02 21:00:00  110.0 1991-01-02 14:00:00  110.0  
...                      ...    ...         

---

In [10]:
def rolled_X_from_2D_sliding_window_df(X_df, timesteps):
    X_rolled = X_df.values.reshape(X_df.shape[0], timesteps, -1)
    return X_rolled

In [11]:
X_rolled_train = rolled_X_from_2D_sliding_window_df(X_df_train, TIME_WINDOW)
X_rolled_valid = rolled_X_from_2D_sliding_window_df(X_df_valid, TIME_WINDOW)
X_rolled_test = rolled_X_from_2D_sliding_window_df(X_df_test, TIME_WINDOW)

In [12]:
print(y_train.shape)
print(X_df_train.shape)
print(X_rolled_train.shape)

print()

print(y_test.shape)
print(X_df_test.shape)
print(X_rolled_test.shape)

(28408,)
(28408, 200)
(28408, 100, 2)

(3439,)
(3439, 200)
(3439, 100, 2)


In [13]:
## Quick data prep that is intuitively a good starting point

VALID_AND_TEST_SIZE = 0.1
TIME_WINDOW=100
FORECAST_DISTANCE=30

targetcol = "BF10"
targetcol_series = df[targetcol]

## train valid test split
X_train, X_else, targetcol_series_train, targetcol_series_else = train_test_split(df[["BF10"]], 
                                   targetcol_series, 
                                   test_size=VALID_AND_TEST_SIZE*2, 
                                   shuffle=False)
X_valid, X_test, targetcol_series_valid, targetcol_series_test = train_test_split(X_else, targetcol_series_else, test_size=0.5, shuffle=False)

## sliding window
X_df_train, y_train = sliding_window_2D_from_df(X_train, targetcol_series_train, TIME_WINDOW, FORECAST_DISTANCE)
X_df_valid, y_valid = sliding_window_2D_from_df(X_valid, targetcol_series_valid, TIME_WINDOW, FORECAST_DISTANCE)
X_df_test, y_test = sliding_window_2D_from_df(X_test, targetcol_series_test, TIME_WINDOW, FORECAST_DISTANCE)

## for lstm
X_rolled_train = rolled_X_from_2D_sliding_window_df(X_df_train, TIME_WINDOW)
X_rolled_valid = rolled_X_from_2D_sliding_window_df(X_df_valid, TIME_WINDOW)
X_rolled_test = rolled_X_from_2D_sliding_window_df(X_df_test, TIME_WINDOW)

print(X_rolled_train.shape)

(28408, 100, 1)


In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras import backend as be
import tensorflow as tf

## we use 10 samples
## and 4 units (hidden state is 4-long for each time step)

tf.random.set_seed(42)
be.clear_session()
input_layer = X_rolled_train[:10]

whole_seq_output, final_memory_state, final_carry_state =  LSTM(4, return_sequences=True, return_state=True)(input_layer)

tf.random.set_seed(42)
be.clear_session()
whole_seq_output2 =  LSTM(4, return_sequences=True, return_state=False)(input_layer)

tf.random.set_seed(42)
be.clear_session()
out_last =  LSTM(4, return_sequences=False, return_state=False)(input_layer)

tf.random.set_seed(42)
be.clear_session()
whole_seq_output3, final_memory_state3, final_carry_state3 =  LSTM(4, return_sequences=False, return_state=True)(input_layer)


print("return_sequences=True, return_state=True")
print(whole_seq_output.shape)
print(final_memory_state.shape)
print(final_carry_state.shape)

print()
print("return_sequences=True, return_state=False")
print(whole_seq_output2.shape)

print()
print("return_sequences=False, return_state=False")
print(out_last.shape)

print()
print("return_sequences=False, return_state=True")
print(whole_seq_output3.shape)
print(final_memory_state3.shape)
print(final_carry_state3.shape)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 300
LEARNING_RATE = 0.1
N_CELLS = 50
KERNEL = 'GlorotNormal'

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

## number of input variables: the final dimension in the 3D prepared data (index 2)
column_count=X_rolled_train.shape[2]

be.clear_session()

model = Sequential()
model.add(LSTM(N_CELLS, activation='tanh', kernel_initializer=KERNEL, return_sequences=True, input_shape = (TIME_WINDOW, column_count)))
model.add(Dropout(0.2))

model.add(LSTM(units = N_CELLS, activation='tanh', kernel_initializer=KERNEL, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = N_CELLS, activation='tanh', kernel_initializer=KERNEL, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = N_CELLS))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

optimizer = Adam(learning_rate=LEARNING_RATE)

## remember, we're doing regression, so choose loss (and output layer) accordingly!
model.compile(loss='mean_squared_error', optimizer='adam')

print(model.summary())

In [ ]:
history = model.fit(x=X_rolled_train,
                    y=y_train, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_rolled_valid,y_valid), 
                    epochs=EPOCHS,
                    verbose=1, 
                    shuffle=False,
                    )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(X_rolled_test).flatten()

In [ ]:
print(predictions[0])

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
 
# Function to create model, required for KerasRegressor
def create_model(optimizer='adam', init='glorot_uniform', activation='tanh' ):
    # create model
    model = Sequential()
    model.add(LSTM(50, activation=activation, kernel_initializer=init, return_sequences=True, input_shape = (TIME_WINDOW, column_count)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, activation=activation, kernel_initializer=init, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, activation=activation, kernel_initializer=init, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))

    model.add(Dense(units = 1))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    return model
 
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

X = X_rolled_train
Y = y_train

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'Adamax', 'SGD']
epochs = [10,20,40]
batches = [16, 32, 100]
activation = ['tanh', 'relu']

param_grid = dict(optimizer=optimizers, epochs=[4], batch_size=batches, activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, verbose=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))